In [1]:
import io
import time

import pandas as pd

from urllib.parse import urlparse
from datetime import datetime, date, UTC
from dateutil.parser import parse
from pymongo import MongoClient, UpdateOne

In [2]:
url = "https://hotnews.ro/sitemap.xml?yyyy=2001&mm=09&dd=21"

In [3]:
parsed = urlparse(url)

parsed

ParseResult(scheme='https', netloc='hotnews.ro', path='/sitemap.xml', params='', query='yyyy=2001&mm=09&dd=21', fragment='')

In [4]:
parsed.query

'yyyy=2001&mm=09&dd=21'

In [5]:
# Extract the date part (remove labels)
formatted_date_str = parsed.query.replace("yyyy=", "").replace("&mm=", "-").replace("&dd=", "-")

# Parse the date using datetime.strptime
date_object = datetime.strptime(formatted_date_str, "%Y-%m-%d")

date_object

datetime.datetime(2001, 9, 21, 0, 0)

In [6]:
date_str = '2024-11-05T05:22:14+00:00'
parse(date_str)

datetime.datetime(2024, 11, 5, 5, 22, 14, tzinfo=tzutc())

In [7]:
datetime.now().isoformat()

'2024-12-16T18:15:32.314510'

### Mongo queries

In [8]:
# Connect to the MongoDB server
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client['crawler']  # Replace with your database name
batches_coll = db['batches']  # Replace with your collection name
cache_coll = db['httpcache']
urls_coll = db['urls_db'] 
content_coll = db['content'] 

In [9]:
# db.getCollectionInfos({'name': content_coll})[0].options.validator

In [10]:
# Query for all documents where 'batch_id' is True
query = {"scraped": False}
documents = list(batches_coll.find(query))

# Iterate and print the documents
for doc in documents:
    print(doc)

ids = [doc['batch_id'] for doc in documents]
ids

{'_id': ObjectId('67487b007fd61c234579f51f'), 'batch_id': 1732803311, 'scraped': False}
{'_id': ObjectId('674e06757fd61c234579f59a'), 'batch_id': 1733166703, 'scraped': False}
{'_id': ObjectId('674ecc657fd61c234579f5cf'), 'batch_id': 1733217371, 'scraped': False}
{'_id': ObjectId('67506c027fd61c234579f5d7'), 'batch_id': 1733323766, 'scraped': False}
{'_id': ObjectId('67506c407fd61c234579f629'), 'batch_id': 1733323806, 'scraped': False}
{'_id': ObjectId('6752d1377fd61c23457a0175'), 'batch_id': 1733480735, 'scraped': False}
{'_id': ObjectId('6752d2627fd61c23457a0b1e'), 'batch_id': 1733481043, 'scraped': False}
{'_id': ObjectId('6752d4f87fd61c23457a0b1f'), 'batch_id': 1733481706, 'scraped': False}
{'_id': ObjectId('6752d5db7fd61c23457a0b20'), 'batch_id': 1733481848, 'scraped': False}
{'_id': ObjectId('675af479647c8c085d4f520c'), 'batch_id': 1734014034, 'scraped': False}


[1732803311,
 1733166703,
 1733217371,
 1733323766,
 1733323806,
 1733480735,
 1733481043,
 1733481706,
 1733481848,
 1734014034]

In [11]:

query = {'batch_id': {'$in': ids}}
projection = {'url': 1, '_id': 0}
# query = {"scraped": False}
documents = list(urls_coll.find(query, projection))
documents =[doc['url'] for doc in documents]
len(documents)

6051

In [12]:
documents[:4]

['https://hotnews.ro/cat-de-periculoasa-ar-fi-victoria-lui-georgescu-si-cat-ar-dura-iesirea-romaniei-din-ue-daca-independentul-ar-ajunge-la-cotroceni-explicatiile-unui-vicepresedinte-al-parlamentului-european-1847441',
 'https://hotnews.ro/cristian-tudor-popescu-indica-o-catastrofa-si-mai-mare-decat-procentul-obtinut-la-alegeri-de-calin-georgescu-cat-a-costat-operatiunea-speciala-de-propagare-pe-tiktok-1847445',
 'https://hotnews.ro/actele-de-sabotaj-ale-rusiei-ar-putea-duce-nato-la-invocarea-articolului-5-spune-seful-serviciului-german-de-spionaj-extern-armata-kremlinului-va-fi-probabil-capabila-sa-atace-nato-pana-la-sfarsitu-1847463',
 'https://hotnews.ro/video-viscolul-secolului-loveste-rusia-autoritatile-au-instituit-stare-de-urgenta-dupa-ce-stratul-de-zapada-a-atins-un-nivel-record-in-regiunea-amur-1847472']

### BRAT

In [13]:
query = {"url": 'https://www.brat.ro/sati/export-rezultate/export/xls/type/site/c/all/period_type/day/category/all/editor/all/regie/all/period_filter/2024-12-3/order_by/name/order/asc/'}
projection = {'url': 1, 'body': 1, '_id': 0}
documents = list(cache_coll.find(query, projection))

len(documents)

1

In [14]:
data = documents[0]['body']

df = pd.read_excel(io.BytesIO(data))
df.columns = map(str.lower, df.columns)
df['tip trafic'] = df['tip trafic'].fillna('total')
df['cat'] = df['categorie'].apply(lambda x: ''.join(i[0:3] for i in x.lower().split()))

df.shape

(776, 12)

In [15]:
df.head()

,nr.,categorie,site,sitecode,tip trafic,editor site,contractor sati,regie de publicitate,afisari,vizite,clienti unici,cat
0,1.0,Auto & moto,www.4tuning.ro,4tuningro,total,Active Soft,Thinkdigital Internet & Advertising SA,Thinkdigital Internet & Advertising SA,28507,12967,11911,aut&mot
1,NaN,Auto & moto,www.4tuning.ro,4tuningro,trafic desktop / laptop,Active Soft,Thinkdigital Internet & Advertising SA,Thinkdigital Internet & Advertising SA,8032,2178,1951,aut&mot
2,NaN,Auto & moto,www.4tuning.ro,4tuningro,trafic mobile (tableta / smartphone),Active Soft,Thinkdigital Internet & Advertising SA,Thinkdigital Internet & Advertising SA,20473,10787,9964,aut&mot
3,NaN,Auto & moto,www.4tuning.ro,4tuningro,aplicatii mobile,Active Soft,Thinkdigital Internet & Advertising SA,Thinkdigital Internet & Advertising SA,0,0,0,aut&mot
4,2.0,Satira & Umor,www.7lucruri.ro,7lucruriro,total,Times New Roman Media SRL,Times New Roman Media SRL,Internet Corp SRL,371,229,210,sat&umo


In [16]:
df['categorie'].nunique(), df['cat'].nunique()

(28, 28)

In [17]:
df_total = df[df['tip trafic'] == 'total']
df_total = df_total.sort_values(by=['afisari'], ascending=False)

df_total.shape

(194, 12)

In [18]:
df_total['categorie'].value_counts()

categorie
Stiri generale                          23
Stiri si analize                        20
Economic & financiar                    19
Lifestyle feminin                       18
Stiri si analize locale                 18
Muzica & Audio                          15
Sanatate & ingrijire personala          11
Sport                                    9
Familie & copii                          8
Culinar                                  8
Auto & moto                              5
Divertisment                             5
Cultura & Arta & Istorie                 4
Stiinta & tehnologie                     4
Filme & cinema                           4
Celebritati si stiri de divertisment     3
IT&C                                     3
Video & TV online                        3
Satira & Umor                            3
Imobiliare                               2
Recomandari de timp liber                2
Comunitati online                        1
Home & deco                              1
S

In [19]:
df_total[:20]

,nr.,categorie,site,sitecode,tip trafic,editor site,contractor sati,regie de publicitate,afisari,vizite,clienti unici,cat
196,50.0,Stiri si analize,www.digi24.ro,digi24ro,total,Digi Romania SA,Digi Romania SA,Digi Romania SA,3897698,1980986,1280842,stisiana
328,83.0,Sport,www.gsp.ro,gspro,total,Ringier Romania SRL,Ringier Romania SRL,Ringier Romania SRL,3676095,830336,541548,spo
204,52.0,Sport,www.digisport.ro,digisportro,total,Digi Romania SA,Digi Romania SA,Digi Romania SA,3546190,1050002,672084,spo
660,166.0,Stiri generale,www.stiripesurse.ro,stiripesursero,total,European Business Environment SRL,European Business Environment SRL,NaN,2439457,621747,396607,stigen
348,88.0,Stiri si analize,www.hotnews.ro,hotnewsro,total,Media Bit Software SRL,ZYX Publishing Group SRL,ZYX Publishing Group SRL,2159301,1155904,765724,stisiana
392,99.0,Stiri generale,www.libertatea.ro,libertatearo,total,Ringier Romania SRL,Ringier Romania SRL,Ringier Romania SRL,2099779,946266,757711,stigen
304,77.0,Stiri si analize,www.g4media.ro,NaN,total,NaN,Thematic Digital SRL,Thematic Digital SRL,1709187,1012691,686254,stisiana
588,148.0,Stiri si analize,www.romaniatv.net,romaniatvnet,total,RTV Properties Management SRL,RTV Properties Management SRL,Thematic Digital SRL,1541003,863248,631065,stisiana
504,127.0,Sport,www.prosport.ro,prosportro,total,Gandul Media Network SRL,Gandul Media Network SRL,Thematic Digital SRL,1419307,744370,545628,spo
656,165.0,Stiri generale,www.stirileprotv.ro,stirileprotvro,total,Pro TV SRL,Pro TV SRL,Pro TV SRL,1333341,858699,673422,stigen


In [29]:
columns = ['site', 'sitecode', 'cat', 'categorie', 'afisari', 'vizite', 'clienti unici']

df_total[columns].head()

,site,sitecode,cat,categorie,afisari,vizite,clienti unici
196,www.digi24.ro,digi24ro,stisiana,Stiri si analize,3897698,1980986,1280842
328,www.gsp.ro,gspro,spo,Sport,3676095,830336,541548
204,www.digisport.ro,digisportro,spo,Sport,3546190,1050002,672084
660,www.stiripesurse.ro,stiripesursero,stigen,Stiri generale,2439457,621747,396607
348,www.hotnews.ro,hotnewsro,stisiana,Stiri si analize,2159301,1155904,765724


In [21]:
df_total.site.unique()

array(['www.digi24.ro', 'www.gsp.ro', 'www.digisport.ro',
       'www.stiripesurse.ro', 'www.hotnews.ro', 'www.libertatea.ro',
       'www.g4media.ro', 'www.romaniatv.net', 'www.prosport.ro',
       'www.stirileprotv.ro', 'www.adevarul.ro', 'www.sport.ro',
       'www.ziare.com', 'www.viva.ro', 'www.cancan.ro', 'www.evz.ro',
       'www.fanatik.ro', 'www.autovit.ro', 'www.antena3.ro',
       'www.mediafax.ro', 'www.pieseauto.ro', 'www.gandul.ro',
       'www.protv.ro', 'www.bugetul.ro', 'www.imobiliare.ro', 'www.zf.ro',
       'www.storia.ro', 'www.sfatulmedicului.ro', 'www.observatornews.ro',
       'www.dcnews.ro', 'www.doctorulzilei.ro', 'www.bucataras.ro',
       'www.unica.ro', 'www.spotmedia.ro', 'www.iamsport.ro',
       'www.click.ro', 'www.newsweek.ro', 'www.capital.ro', 'www.a1.ro',
       'www.economica.net', 'www.libertateapentrufemei.ro', 'www.tpu.ro',
       'www.csid.ro', 'www.profit.ro', 'www.edupedu.ro',
       'www.cotidianul.ro', 'www.ebihoreanul.ro', 'www.kanald.ro'

In [30]:
df_total.categorie.unique()

array(['Stiri si analize', 'Sport', 'Stiri generale', 'Lifestyle feminin',
       'Auto & moto', 'Divertisment', 'Economic & financiar',
       'Imobiliare', 'Sanatate & ingrijire personala', 'Culinar',
       'Comunitati online', 'Stiri si analize locale',
       'Celebritati si stiri de divertisment', 'Stiinta & tehnologie',
       'Muzica & Audio', 'Filme & cinema', 'Familie & copii',
       'Satira & Umor', 'Home & deco', 'Cultura & Arta & Istorie',
       'Video & TV online', 'IT&C', 'Spiritualitate',
       'Portaluri & motoare de cautare', 'Educatie',
       'Recomandari de timp liber', 'Hobby', 'Altele'], dtype=object)

In [41]:
df_total[df_total['categorie'] == 'Video & TV online']

,nr.,categorie,site,sitecode,tip trafic,editor site,contractor sati,regie de publicitate,afisari,vizite,clienti unici,cat
28,8.0,Video & TV online,www.alephnews.ro,alephnewsro,total,Aleph Media SRL,Outsmart Media SRL,Outsmart Media SRL,24246,18502,17202,vid&tvonl
24,7.0,Video & TV online,www.alephbusiness.ro,alephbusiness.ro,total,Aleph Media SRL,Outsmart Media SRL,Outsmart Media SRL,593,300,245,vid&tvonl
624,157.0,Video & TV online,www.smartv.media,smartvmedia,total,Aleph Media SRL,Outsmart Media SRL,Outsmart Media SRL,179,155,151,vid&tvonl


In [42]:
exclude_list = ['Comunitati online', 'Imobiliare', 'Portaluri & motoare de cautare', 'Altele', 'Filme & cinema']

### Create collection to store websites

In [22]:
df_total.columns

Index(['nr.', 'categorie', 'site', 'sitecode', 'tip trafic', 'editor site',
       'contractor sati', 'regie de publicitate', 'afisari', 'vizite',
       'clienti unici', 'cat'],
      dtype='object')

In [23]:
exclude = ['nr.', 'tip trafic', 'afisari', 'vizite', 'clienti unici', ]

exclude = ['nr.', 'tip trafic', 'vizite', 'clienti unici', ]
websites = df_total[df_total.columns[~df_total.columns.isin(exclude)]]

print(websites.shape)
websites.head(2)

(194, 8)


,categorie,site,sitecode,editor site,contractor sati,regie de publicitate,afisari,cat
196,Stiri si analize,www.digi24.ro,digi24ro,Digi Romania SA,Digi Romania SA,Digi Romania SA,3897698,stisiana
328,Sport,www.gsp.ro,gspro,Ringier Romania SRL,Ringier Romania SRL,Ringier Romania SRL,3676095,spo


In [24]:
unique_sites = db['brat'].distinct("site")

len(unique_sites)

195

In [25]:
unique_sites

['www.4tuning.ro',
 'www.7lucruri.ro',
 'www.a1.ro',
 'www.adevarul.ro',
 'www.agrointel.ro',
 'www.alba24.ro',
 'www.alephbusiness.ro',
 'www.alephnews.ro',
 'www.ambasadoriistiintei.ro',
 'www.antena3.ro',
 'www.antenasatelor.ro',
 'www.antenastars.ro',
 'www.as.ro',
 'www.automarket.ro',
 'www.autovit.ro',
 'www.avantaje.ro',
 'www.avocatnet.ro',
 'www.b1tv.ro',
 'www.b365.ro',
 'www.bizbrasov.ro',
 'www.bucataras.ro',
 'www.bugetul.ro',
 'www.businessmagazin.ro',
 'www.canal33.ro',
 'www.cancan.ro',
 'www.capital.ro',
 'www.catchy.ro',
 'www.catine.ro',
 'www.ciao.ro',
 'www.cinemagia.ro',
 'www.click.ro',
 'www.clickpentrufemei.ro',
 'www.clickpoftabuna.ro',
 'www.clicksanatate.ro',
 'www.clubulbebelusilor.ro',
 'www.conso.ro',
 'www.cotidianul.ro',
 'www.crisana.ro',
 'www.csid.ro',
 'www.culinar.ro',
 'www.cursdeguvernare.ro',
 'www.dailybusiness.ro',
 'www.dcnews.ro',
 'www.defapt.ro',
 'www.deparinti.ro',
 'www.descopera.ro',
 'www.desprecopii.com',
 'www.digi-animalworld.tv',

In [26]:
ops = []

for idx, row in websites.iterrows():
    row = row.to_dict()
    
    ops.append(UpdateOne({'site': row.pop('site', None)}, {'$set': row}, upsert=True))

print(len(ops))
    
result = db['websites'].bulk_write(ops)

194


In [27]:
# result

In [28]:
ops[0]

UpdateOne({'site': 'www.digi24.ro'}, {'$set': {'categorie': 'Stiri si analize', 'sitecode': 'digi24ro', 'editor site': 'Digi Romania SA', 'contractor sati': 'Digi Romania SA', 'regie de publicitate': 'Digi Romania SA', 'afisari': 3897698, 'cat': 'stisiana'}}, True, None, None, None)

In [35]:
df_total[:20].site.tolist()

['www.digi24.ro',
 'www.gsp.ro',
 'www.digisport.ro',
 'www.stiripesurse.ro',
 'www.hotnews.ro',
 'www.libertatea.ro',
 'www.g4media.ro',
 'www.romaniatv.net',
 'www.prosport.ro',
 'www.stirileprotv.ro',
 'www.adevarul.ro',
 'www.sport.ro',
 'www.ziare.com',
 'www.viva.ro',
 'www.cancan.ro',
 'www.evz.ro',
 'www.fanatik.ro',
 'www.autovit.ro',
 'www.antena3.ro',
 'www.mediafax.ro']